In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import os

folder_path = '/content/drive/MyDrive/WISDM---Biometric-time-series-data-classification-main/Datasets/train'

files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.txt')]

data_frames = [pd.read_csv(file, header=None, names=['user', 'activity', 'x-accel', 'y-accel', 'z-accel'], sep=',', on_bad_lines='skip') for file in files]
combined_data = pd.concat(data_frames, ignore_index=True)

print(combined_data.head())


   user activity   x-accel   y-accel     z-accel
0  1600        A  7.091625 -0.591667   8.195502;
1  1600        A  4.972757 -0.158317  6.6967316;
2  1600        A  3.253720 -0.191835   6.107758;
3  1600        A  2.801216 -0.155922   5.997625;
4  1600        A  3.770868 -1.051354   7.731027;


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/WISDM---Biometric-time-series-data-classification-main/Datasets/test/test_data_accel_watch.txt'  # Adjust the path and file name accordingly

test_data = pd.read_csv(file_path, header=None, names=[ 'user', 'x-accel', 'y-accel', 'z-accel'], sep=',',on_bad_lines='skip')
print(test_data.head())



   user   x-accel   y-accel     z-accel
0  1650  0.003142 -2.910451   9.310901;
0  1650  0.024690 -2.735675    9.41864;
0  1650 -0.068684 -2.862567   9.409063;
0  1650 -0.032771 -2.728492  9.5910225;
0  1650  0.094122 -2.903269   9.365968;


In [ ]:
def clean_data(data):
    for col in ['x-accel', 'y-accel', 'z-accel']:
        data[col] = data[col].astype(str).str.replace(';', '').astype(float)
    return data

combined_data = clean_data(combined_data)
test_data = clean_data(test_data)

Logisitc regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = combined_data[['x-accel', 'y-accel', 'z-accel']]
y = combined_data['activity']
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=1000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the logistic regression model on the test set: {accuracy * 100:.2f}%")


Accuracy of the logistic regression model on the test set: 31.90%


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X_train = combined_data[['x-accel', 'y-accel', 'z-accel']]
y_train = combined_data['activity']
X_train_imputed = imputer.fit_transform(X_train)
X_train_scaled = scaler.fit_transform(X_train_imputed)

X_test = test_data[['x-accel', 'y-accel', 'z-accel']]
X_test_imputed = imputer.transform(X_test)
X_test_scaled = scaler.transform(X_test_imputed)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

print("Predicted Activities:")
print(y_pred)


Predicted Activities:
['D' 'D' 'D' ... 'B' 'B' 'B']


Decision Tree

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = combined_data[['x-accel', 'y-accel', 'z-accel']]
y = combined_data['activity']
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

decision_tree_model = DecisionTreeClassifier(random_state=42)

decision_tree_model.fit(X_train, y_train)

y_pred = decision_tree_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree model on the test set: {accuracy * 100:.2f}%")


Accuracy of the Decision Tree model on the test set: 53.88%


Random Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = combined_data[['x-accel', 'y-accel', 'z-accel']]
y = combined_data['activity']
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest_model.fit(X_train, y_train)

y_pred = random_forest_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Random Forest model on the test set: {accuracy * 100:.2f}%")


Accuracy of the Random Forest model on the test set: 60.12%


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = combined_data[['x-accel', 'y-accel', 'z-accel']]
y = combined_data['activity']
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)
X_scaled = np.expand_dims(X_scaled, axis=2)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)


model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy of the 1D CNN model on the test set: {accuracy * 100:.2f}%')


Epoch 1/10
16275/16275 [==============================] - 43s 3ms/step - loss: 1.2339 - accuracy: 0.5026 - val_loss: 1.1222 - val_accuracy: 0.5547
Epoch 2/10
16275/16275 [==============================] - 72s 4ms/step - loss: 1.1520 - accuracy: 0.5414 - val_loss: 1.1221 - val_accuracy: 0.5551
Epoch 3/10
16275/16275 [==============================] - 57s 4ms/step - loss: 1.1314 - accuracy: 0.5494 - val_loss: 1.1103 - val_accuracy: 0.5624
Epoch 4/10
16275/16275 [==============================] - 60s 4ms/step - loss: 1.1213 - accuracy: 0.5530 - val_loss: 1.0977 - val_accuracy: 0.5623
Epoch 5/10
16275/16275 [==============================] - 65s 4ms/step - loss: 1.1139 - accuracy: 0.5564 - val_loss: 1.0941 - val_accuracy: 0.5681
Epoch 6/10
16275/16275 [==============================] - 54s 3ms/step - loss: 1.1090 - accuracy: 0.5585 - val_loss: 1.0941 - val_accuracy: 0.5662
Epoch 7/10
16275/16275 [==============================] - 42s 3ms/step - loss: 1.1056 - accuracy: 0.5592 - val_loss: 1

In [ ]:
from tensorflow.keras.layers import Conv1D, Input, Dense, Flatten, concatenate, MaxPooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np

input_layer = Input(shape=(X_train.shape[1], 1))


conv1 = Conv1D(filters=16, kernel_size=5, activation='elu', padding='same')(input_layer)
conv1 = MaxPooling1D(pool_size=2)(conv1)

conv2 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)
conv2 = MaxPooling1D(pool_size=2)(conv2)

conv3 = Conv1D(filters=16, kernel_size=3, activation='elu', padding='same')(input_layer)
conv3 = MaxPooling1D(pool_size=3)(conv3)

merged = concatenate([conv1, conv2, conv3])

conv4 = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(merged)

conv5 = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(conv4)

conv6 = Conv1D(filters=32, kernel_size=5, activation='relu', padding='same')(conv5)


flatten = Flatten()(conv6)
dense = Dense(256, activation='relu')(flatten)
output_layer = Dense(y_train.shape[1], activation='softmax')(dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test), verbose=1)

_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy of the 1D CNN model on the test set: {accuracy * 100:.2f}%')


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 3, 1)]               0         []                            
                                                                                                  
 conv1d_33 (Conv1D)          (None, 3, 16)                96        ['input_9[0][0]']             
                                                                                                  
 conv1d_34 (Conv1D)          (None, 3, 32)                128       ['input_9[0][0]']             
                                                                                                  
 conv1d_35 (Conv1D)          (None, 3, 16)                64        ['input_9[0][0]']             
                                                                                            

In [ ]:
from tensorflow.keras.layers import Conv1D, Input, Dense, Flatten, concatenate, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

input_layer = Input(shape=(X_train.shape[1], 1))

conv1 = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(input_layer)
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)

merged = concatenate([conv1, conv2])

conv3 = Conv1D(filters=16, kernel_size=3, activation='elu', padding='same')(merged)

flatten = Flatten()(conv3)
dense = Dense(256, activation='relu')(flatten)
output_layer = Dense(y_categorical.shape[1], activation='softmax')(dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy of the 1D CNN model on the test set: {accuracy * 100:.2f}%')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 3, 1)]               0         []                            
                                                                                                  
 conv1d_30 (Conv1D)          (None, 3, 16)                96        ['input_13[0][0]']            
                                                                                                  
 conv1d_31 (Conv1D)          (None, 3, 64)                256       ['input_13[0][0]']            
                                                                                                  
 concatenate_8 (Concatenate  (None, 3, 80)                0         ['conv1d_30[0][0]',           
 )                                                                   'conv1d_31[0][0]']     

CNN+LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Flatten, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X = combined_data[['x-accel', 'y-accel', 'z-accel']].values
y = combined_data['activity'].values

X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)
X_scaled = np.expand_dims(X_scaled, axis=2)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)

input_layer = Input(shape=(X_train.shape[1], 1))

conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
bilstm_layer = Bidirectional(LSTM(50, return_sequences=False))(conv_layer)
flatten_layer = Flatten()(bilstm_layer)
output_layer = Dense(y_categorical.shape[1], activation='softmax')(flatten_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

_, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy of the CNN+BiLSTM model: {accuracy * 100:.2f}%')

feature_model = Model(inputs=model.input, outputs=model.layers[-3].output)
features_train = feature_model.predict(X_train)
features_test = feature_model.predict(X_test)


Epoch 1/20
16275/16275 [==============================] - 131s 8ms/step - loss: 1.0773 - accuracy: 0.5688 - val_loss: 1.0240 - val_accuracy: 0.5895
Epoch 2/20
16275/16275 [==============================] - 130s 8ms/step - loss: 1.0068 - accuracy: 0.5945 - val_loss: 0.9938 - val_accuracy: 0.5991
Epoch 3/20
16275/16275 [==============================] - 123s 8ms/step - loss: 0.9919 - accuracy: 0.5990 - val_loss: 0.9847 - val_accuracy: 0.6021
Epoch 4/20
16275/16275 [==============================] - 124s 8ms/step - loss: 0.9845 - accuracy: 0.6013 - val_loss: 0.9790 - val_accuracy: 0.6045
Epoch 5/20
16275/16275 [==============================] - 123s 8ms/step - loss: 0.9787 - accuracy: 0.6036 - val_loss: 0.9743 - val_accuracy: 0.6036
Epoch 6/20
16275/16275 [==============================] - 126s 8ms/step - loss: 0.9748 - accuracy: 0.6039 - val_loss: 0.9744 - val_accuracy: 0.6054
Epoch 7/20
16275/16275 [==============================] - 129s 8ms/step - loss: 0.9714 - accuracy: 0.6058 - val_

In [ ]:
from tensorflow.keras.layers import Conv1D, Input, Dense, Flatten, concatenate, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

input_layer = Input(shape=(X_train.shape[1], 1))

conv1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)
conv2 = Conv1D(filters=64, kernel_size=3, activation='elu', padding='same')(input_layer)
conv3 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)

merged = concatenate([conv1, conv2,conv3])

conv4 = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(merged)
conv5 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(input_layer)
conv6 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)

flatten = Flatten()(conv3)
dense = Dense(256, activation='relu')(flatten)
output_layer = Dense(y_categorical.shape[1], activation='softmax')(dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=1)

_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy of the 1D CNN model on the test set: {accuracy * 100:.2f}%')


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 conv1d_9 (Conv1D)           (None, 3, 32)             128       
                                                                 
 flatten_2 (Flatten)         (None, 96)                0         
                                                                 
 dense_3 (Dense)             (None, 256)               24832     
                                                                 
 dense_4 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 26502 (103.52 KB)
Trainable params: 26502 (103.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
16275/16275 